#  Science Image Calibration notebook

This notebook calibrates science images by removing bias, dark and dividing out flats.

Made by: Harlan Shaw <harlan.shaw@ucalgary.ca>

## Required Python packages:
[Astropy](https://www.astropy.org/)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/index.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)

It's recommended you download and install [Anaconda](https://www.anaconda.com/products/individual#Downloads) as this contains a Python environment and Astropy.

You will need to install CCDProc using the Conda install command: `conda install -c conda-forge ccdproc`

This will also install Astropy and Astro-SCRAPPY if not already installed.

## Citations:
This project uses:

[Astropy](https://www.astropy.org/acknowledging.html)

[CCDProc](https://ccdproc.readthedocs.io/en/latest/citation.html)

[Astro-SCRAPPY](https://github.com/astropy/astroscrappy)


## Required Variables

The next cell has variables that must be set for the notebook to function

In [ ]:
from pathlib import Path
from astropy.wcs import FITSFixedWarning
import warnings
import numpy as np

warnings.filterwarnings("ignore", category=FITSFixedWarning)
# Replace each point in Path() with the bias, dark, flat, mask files and data directory as below
DATA_DIRECTORY = Path("C:/Baker-Nunn")
PATH_TO_BIAS = DATA_DIRECTORY / "Calibration" / "combined_bias_5C_mean.fit"
PATH_TO_DARK = DATA_DIRECTORY / "Calibration" / "combined_dark_12C_20s_mean.fit"
PATH_TO_FLAT = DATA_DIRECTORY / "Calibration" / "combined_flat_median_12c.fit"
PATH_TO_MASK = DATA_DIRECTORY / "Calibration" / "combined_mask.fit"
PATH_TO_DATA = DATA_DIRECTORY / "2022July20_TrES-2b_transit_bn" / "Lights"

# Handy functions for replacing bad pixels

def clamp(value, min, max):
    """Clamps value to specified range"""
    return (min if value < min else max if value > max else value)

def get_surrounding_square(array, idx, len_from_idx):
    """Returns the surrounding square of len_from_idx pixels from the x-y index provided in the array, if anything.
    For example, if I give the index (1000, 2500) and the len_from_idx 5, it will give me everything from (995, 2495) to
    (1005, 2505)
    """
    max_x_index = array.shape[0]
    max_y_index = array.shape[1]

    min_x = clamp(idx[0] - len_from_idx, 0, max_x_index)
    min_y = clamp(idx[1] - len_from_idx, 0, max_y_index)

    # +1 so we can get the last value in the x/y since
    # Python's slicing stop is not included.
    max_x = clamp(idx[0] + len_from_idx, 0, max_x_index) + 1
    max_y = clamp(idx[1] - len_from_idx, 0, max_y_index) + 1

    return array[min_x:max_x, min_y:max_y]

def replace_mask_with_median(array, mask, square: int = 9):
    """Goes over every x, y coordinate flagged as 1 (Bad) in a mask
    then replaces that pixel's value with the median of the square around it
    by default 9 pixels from the centre in each direction"""
    indices = map(tuple, np.nonzero(mask))

    for index in indices:
        square = get_surrounding_square(array, index, square)
        median = np.median(square)

        np.put(array, [index[0], index[1]], median)

    return array


In [ ]:
from pathlib import Path
from astropy.nddata import CCDData
from astropy.units import adu, second, dimensionless_unscaled
from ccdproc import ImageFileCollection, subtract_bias, subtract_dark, flat_correct, cosmicray_lacosmic
from astropy.convolution import Gaussian2DKernel, interpolate_replace_nans

data_files = ImageFileCollection(PATH_TO_DATA, glob_include="*.fit*")
# Read in all fits files
combined_bias = CCDData.read(PATH_TO_BIAS, unit=adu)
combined_dark = CCDData.read(PATH_TO_DARK, unit=adu)
combined_flat = CCDData.read(PATH_TO_FLAT, unit=adu)
combined_mask = CCDData.read(PATH_TO_MASK, unit=dimensionless_unscaled)
# Take mask matrix out of CCD
mask = combined_mask.data
# Set up output directory
output_directory = PATH_TO_DATA.joinpath("corrected_b_d_f")
output_directory.mkdir(parents=True, exist_ok=True)
# To help remove bad pixels by replacing them with nearby data
kernel = Gaussian2DKernel(x_stddev=2)

for file_name in data_files.files:
    file_name_stem = Path(file_name).stem
    full_path = PATH_TO_DATA.joinpath(file_name)
    # Since we're doing 3 modifications, output filename will be _b_d_f
    # for bias, dark and flat respectively
    new_filename = full_path.with_stem(file_name_stem + "_b_d_f")
    output_file = output_directory.joinpath(new_filename.name)
    # Read in datafile and apply changes
    data_ccd = CCDData.read(full_path, unit=adu)

    # Standard calibration
    data_ccd = subtract_bias(data_ccd, combined_bias, add_keyword={"HISTORY": f"Subtracted master bias {PATH_TO_BIAS.name}"})
    data_ccd = subtract_dark(ccd=data_ccd, master=combined_dark, exposure_time="EXPTIME", exposure_unit=second, scale=True, add_keyword={"HISTORY": f"Subtracted master dark {PATH_TO_DARK.name}"})
    data_ccd = flat_correct(ccd=data_ccd, flat=combined_flat, add_keyword={"HISTORY": f"Divided master flat {PATH_TO_FLAT.name}"})

    # Improving the final result by finding cosmic rays to filter out
    _, new_mask = cosmicray_lacosmic(ccd=data_ccd.data, sigclip=8, readnoise=14.7, gain=1.42, gain_apply=True)
    new_mask = new_mask | mask # | is logical OR. if any values are 1 (True, bad pixels) then they are preserved

    # Set to not a number so we can use the interpolate function

    data_ccd.data[new_mask] = np.nan

    data_ccd.data = interpolate_replace_nans(data_ccd.data, kernel)

    data_ccd.header["HISTORY"] = "Replaced all bad pixels with 17x17 Gaussian2d"
    data_ccd.write(output_file, output_verify="ignore")
